In [ ]:
import requests
import numpy as np
import pandas as pd
import re
import os

from dataclasses import dataclass, field
from typing import Optional, List
from datetime import datetime, timedelta

import matplotlib as mpl
import matplotlib.pyplot as plt

from cycler import cycler

In [ ]:
# mpl.use("pdf")
# plt.close("all")
plt.rcParams.update(
    {
        "text.usetex": False,
        "font.family": "serif",
        "pgf.texsystem": "pdflatex",
        "font.size": 12,  # footnote/caption size 9pt for paper
        # "font.size": 10,     # caption size 10pt on thesis
        "pgf.preamble": "\n".join(
            [
                r"\usepackage{libertine}",
                # r"\usepackage{lmodern}",
            ]
        ),
        # "lines.linewidth": 0.8,
        "lines.markersize": 3,
        "axes.linewidth": 0.5,
        "grid.linewidth": 0.3,
        "grid.linestyle": "-",
        "axes.edgecolor": mpl.rcParams["grid.color"],
        # "ytick.color": mpl.rcParams["grid.color"],
        "ytick.direction": "in",
        # "xtick.color": mpl.rcParams["grid.color"],
        "xtick.direction": "in",
        "axes.titlesize": "medium",
        "axes.titlepad": 4,
        "axes.labelpad": 1,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.spines.bottom": False,
        "axes.spines.left": False,
        "axes.axisbelow": True,  # grid below patches
        "axes.prop_cycle": cycler(
            "color", ["#348ABD", "#7A68A6", "#A60628", "#467821", "#CF4457", "#188487", "#E24A33"]
        ),
        "legend.labelspacing": 0.1,
        "legend.handlelength": 1,
        "legend.handletextpad": 0.2,
        "legend.columnspacing": 1,
        "legend.borderpad": 0.1,
    }
)


In [ ]:
@dataclass
class AxisConfig:
    labels: list[str] = field(default_factory=list)
    ylabel: str = None
    ylim: tuple[float, float] = None
    plot_kwargs: list[dict] = field(default_factory=list)  # List of dicts for each series' styling


@dataclass
class CommonPlotConfig:
    title: str
    xlim: tuple[float, float] = None
    figsize: tuple[int, int] = (10, 6)
    show_legend: bool = True
    legend_kwargs: dict = field(default_factory=dict)
    tight_layout: bool = True
    grid: bool = True
    grid_minor: bool = True
    grid_kwargs: dict = field(default_factory=dict)
    minor_grid_kwargs: dict = field(default_factory=dict)
    minor_ticks: bool = True
    subplots_adjust: dict = None
    time_unit: str = 's'

@dataclass
class PlotConfig:
    common: CommonPlotConfig
    primary_axis: AxisConfig
    secondary_axis: AxisConfig = None

In [ ]:
#NOT USED

# PROMETHEUS_URL = "http://localhost:40333/api/v1"

# @dataclass
# class PrometheusQueryConfig:
#     prometheus_url: str
#     query: str
#     start: float
#     end: float
#     step: str = '1s'

# def list_all_metrics(prometheus_url):
#     response = requests.get(f"{prometheus_url}/label/__name__/values")
#     if response.status_code == 200:
#         return response.json()['data']
#     else:
#         raise Exception(f"Error fetching metrics list: {response.status_code}, {response.text}")

# def query_prometheus_range(config: PrometheusQueryConfig):
#     params = {
#         'query': config.query,
#         'start': config.start,
#         'end': config.end,
#         'step': config.step
#     }
#     response = requests.get(f"{config.prometheus_url}/query_range", params=params)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         raise Exception(f"Error querying Prometheus: {response.status_code}, {response.text}")

# def to_df(prometheus_response) -> list[pd.DataFrame]:
#     time_series = len(prometheus_response['data']['result'])
#     if time_series == 0:
#         raise Exception("No data found")
#     dfs = []
#     for data in prometheus_response['data']['result']:
#         df = pd.DataFrame(data['values'], columns=['timestamp', 'value'])
#         df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
#         df['value'] = df['value'].astype(float)
#         dfs.append(df)
#     return dfs

# def plot_query(
#     primary_query_config: PrometheusQueryConfig,
#     secondary_query_config: Optional[PrometheusQueryConfig],
#     plot_config: PlotConfig
# ):
    
#     primary_response = query_prometheus_range(primary_query_config)
#     primary_dfs = to_df(primary_response)

#     secondary_dfs = None
#     if secondary_query_config:
#         secondary_response = query_prometheus_range(secondary_query_config)
#         secondary_dfs = to_df(secondary_response)
    
#     plot_df(primary_dfs, secondary_dfs, plot_config)

In [ ]:
def plot_df(
    primary_dfs: List[pd.DataFrame],
    secondary_dfs: Optional[List[pd.DataFrame]],
    plot_config: PlotConfig
):
    # Extract configurations from plot_config
    common = plot_config.common
    primary_axis = plot_config.primary_axis
    secondary_axis = plot_config.secondary_axis
    title = common.title
    time_unit = common.time_unit
    show_legend = common.show_legend
    grid = common.grid
    grid_minor = common.grid_minor
    tight_layout = common.tight_layout

    # Calculate the minimum timestamp for alignment
    min_time = min(
        [df['timestamp'].min() for df in primary_dfs + (secondary_dfs or [])]
    )

    # Plotting
    fig, ax1 = plt.subplots(figsize=common.figsize)

    # Plot primary y-axis data
    for i, df in enumerate(primary_dfs):
        df['time_since_start'] = (df['timestamp'] - min_time).dt.total_seconds()
        plot_kwargs = primary_axis.plot_kwargs[i] if i < len(primary_axis.plot_kwargs) else {}
        ax1.plot(
            df['time_since_start'], df['value'], 
            label=primary_axis.labels[i] if primary_axis.labels else None,
            **plot_kwargs  # Apply unique styling options per time series
        )

    # Configure secondary y-axis if provided
    if secondary_axis and secondary_dfs:
        ax2 = ax1.twinx()
        for i, df in enumerate(secondary_dfs):
            df['time_since_start'] = (df['timestamp'] - min_time).dt.total_seconds()
            plot_kwargs = secondary_axis.plot_kwargs[i] if i < len(secondary_axis.plot_kwargs) else {}
            ax2.plot(
                df['time_since_start'], df['value'],
                label=secondary_axis.labels[i] if secondary_axis.labels else None,
                **plot_kwargs  # Apply unique styling options per time series on the secondary axis
            )
        ax2.set_ylabel(secondary_axis.ylabel)
        if secondary_axis.ylim:
            ax2.set_ylim(secondary_axis.ylim)
        
    # Set axis labels and limits
    ax1.set_xlabel(f"Time ({time_unit})")
    ax1.set_ylabel(primary_axis.ylabel)
    if primary_axis.ylim:
        ax1.set_ylim(primary_axis.ylim)
    if common.xlim:
        ax1.set_xlim(common.xlim)
        if secondary_axis and secondary_dfs:
            ax2.set_xlim(common.xlim)

    # Plot title and legends
    plt.title(title)
    if show_legend:
        lines_primary, labels_primary = ax1.get_legend_handles_labels()
        if secondary_axis and secondary_dfs:
            lines_secondary, labels_secondary = ax2.get_legend_handles_labels()
            ax1.legend(lines_primary + lines_secondary, labels_primary + labels_secondary, **common.legend_kwargs)
        else:
            ax1.legend(lines_primary, labels_primary, **common.legend_kwargs)

    # Grid and layout options
    if common.minor_ticks:
        ax1.minorticks_on()
        if secondary_axis and secondary_dfs:
            ax2.minorticks_on()
    if grid:
        ax1.grid(visible=True, which='both', axis='both', **common.grid_kwargs)
    if grid_minor:
        ax1.grid(visible=True, which='minor', axis='both', **common.minor_grid_kwargs)

    if secondary_axis and secondary_dfs:
        # Align the secondary y-axis with the primary y-axis
        ax2.set_yticks(np.linspace(ax2.get_yticks()[0], ax2.get_yticks()[-1], len(ax1.get_yticks())))
    
    if tight_layout:
        plt.tight_layout()
    if common.subplots_adjust:
        plt.subplots_adjust(**common.subplots_adjust)

    plt.show()

In [ ]:
# NOT USED

# def to_range(end: str, duration: str) -> tuple[float, float]:
#     # Convert the date format to timestamp in unix epoch
#     end_datetime = datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
#     end_timestamp = end_datetime.timestamp()
    
#     # Extract the duration components using regex
#     duration_match = re.match(r"(?:(\d+)m)?(?:(\d+)s)?(?:(\d+)ms)?", duration)
#     minutes = int(duration_match.group(1)) if duration_match.group(1) else 0
#     seconds = int(duration_match.group(2)) if duration_match.group(2) else 0
#     milliseconds = int(duration_match.group(3)) if duration_match.group(3) else 0

#     # Calculate the duration
#     duration_delta = timedelta(minutes=minutes, seconds=seconds, milliseconds=milliseconds)

#     # Calculate the start timestamp
#     start_timestamp = (end_datetime - duration_delta).timestamp()
    
#     return end_timestamp, start_timestamp

# end="2024-11-14 12:35:29"
# duration="37m57s472ms"

# END, START = to_range(end, duration)

# # primary_query = PrometheusQueryConfig(
# #     prometheus_url=PROMETHEUS_URL,
# #     query="rate(container_cpu_usage_seconds_total{pod=~'multipaper-server-0|multipaper-server-1',container!=''}[5m])",
# #     start=START,
# #     end=END,
# #     step='1s'
# # )

# # primary_axis = AxisConfig(
# #     labels=["Server 0 CPU", "Server 1 CPU"],
# #     ylabel="CPU Usage (\%)",
# #     ylim=(0, 4),
# #     plot_kwargs=[
# #         {"color": "red", "linestyle": "-"},
# #         {"color": "blue", "linestyle": "-"}
# #     ]
# # )

# primary_query = PrometheusQueryConfig(
#     prometheus_url=PROMETHEUS_URL,
#     query="mc_players_online_local{world='world'}",
#     start=START,
#     end=END,
#     step='1s'
# )

# primary_axis = AxisConfig(
#     labels=["Server 0 Players", "Server 1 Players", "Server 2 Players", "Server 3 Players"],
#     ylabel="Players",
#     ylim=(0, 300),
#     plot_kwargs=[
#         {"color": "red", "linestyle": "-"},
#         {"color": "blue", "linestyle": "-"},
#         {"color": "green", "linestyle": "-"},
#         {"color": "orange", "linestyle": "-"}
#     ]
# )


# secondary_query = PrometheusQueryConfig(
#     prometheus_url=PROMETHEUS_URL,
#     query="mc_tps",
#     start=START,
#     end=END,
#     step='1s'
# )
# secondary_axis = AxisConfig(
#     labels=["Server 0 TPS", "Server 1 TPS", "Server 2 TPS", "Server 3 TPS"],
#     ylabel="TPS",
#     ylim=(0, 20),
#     plot_kwargs=[
#         {"color": "red", "linestyle": "--"},
#         {"color": "blue", "linestyle": "--"},
#         {"color": "green", "linestyle": "--"},
#         {"color": "orange", "linestyle": "--"}
#     ]
# )

# common_conf = CommonPlotConfig(
#     title="CPU Usage and TPS",
#     figsize=(15, 6),
#     show_legend=True,
#     legend_kwargs={"loc": "upper center"},
#     tight_layout=True,
#     grid=True,
#     grid_minor=False,
#     minor_ticks=False,
#     # grid_kwargs={"linestyle": "-"},
#     # minor_grid_kwargs={"linestyle": ":"},
#     time_unit='s'
# )

# conf = PlotConfig(
#     common=common_conf,
#     primary_axis=primary_axis,
#     secondary_axis=secondary_axis
# )

# plot_query(primary_query, secondary_query, conf)

In [ ]:
def save_metrics(selected_metrics, from_time, to_time, source, extractor, experiment):
    metrics_data = extractor.extract(
        source=source,
        metrics=selected_metrics,
        from_time=from_time,
        to_time=to_time,
        output_format="pandas",
    )
    # create the directory if it does not exist
    output_dir = f"metrics/{experiment}"
    os.makedirs(output_dir, exist_ok=True)
    # save the data to a file
    # for metric_name, data in metrics_data.items():
    #     # only get the name of the metric, remove the filter part or the function part
    #     metric_name = metric_name.split("{")[0]
    #     # if it contains a function, remove the function part
    #     if "(" in metric_name:
    #         metric_name = metric_name.split("(")[1]
    #     data.to_csv(f"metrics/{experiment}/{metric_name}.csv")

    for metric_name, data in metrics_data.items():
        # only get the name of the metric, remove the filter part or the function part
        base_metric_name = metric_name.split("{")[0]
        # if it contains a function, remove the function part
        # if "(" in base_metric_name:
        #     base_metric_name  = base_metric_name.split("(")[1]

        # add a suffix to the metric name
        suffix = ""
        if "master" in metric_name:
            suffix = "_master"
        elif "server" in metric_name:
            suffix = "_server"

        data.to_csv(f"metrics/{experiment}/{base_metric_name}{suffix}.csv")


In [ ]:
from metrics_extractor import MetricsExtractor, PrometheusSource

# 27 metrics
selected_metrics = [
    "mc_players_online_local{world='world'}",
    "mc_tps",
    "sum(mc_tps)",
    "sum(mc_players_online_local{world='world'})",
    "count(count by(server_name) (mc_tps))",
    "mc_mspt_seconds_10_mean",
    "minecraft_tick_duration_seconds_bucket",
    "sum(rate(minecraft_tick_duration_seconds_bucket[2s])) by(le)",
    "rate(container_network_receive_bytes_total{pod=~'multipaper-master-.*'}[1m])",
    "rate(container_network_transmit_bytes_total{pod=~'multipaper-master-.*'}[1m])",
    "rate(container_network_receive_bytes_total{pod=~'multipaper-server-.*'}[1m])",
    "rate(container_network_transmit_bytes_total{pod=~'multipaper-server-.*'}[1m])",
    "container_memory_usage_bytes{container='multipaper-master-velocity'}",
    "container_memory_usage_bytes{container='multipaper-server'}",
    "rate(container_cpu_usage_seconds_total{container='multipaper-master-velocity'}[1m])",
    "rate(container_cpu_usage_seconds_total{container='multipaper-server'}[1m])", 
    "mc_chunk_ownership",
    "sum by(owner) (mc_chunk_ownership)", 
    "minecraft_world_loaded_chunks{world='world'}",
    "mc_player_location",
    "sum by(chunk_owner) (mc_player_location)",
    "server_latency",
    "avg(server_latency)",
    "max(server_latency)",
    "min(server_latency)",
    "quantile(0.5, server_latency)",
    "quantile(0.99, server_latency)"
]

experiment = "2s_pvp_batch"

input_time = {"from":"2025-05-22 10:26:43","to":"2025-05-22 10:39:10"}

from_time = datetime.strptime(input_time["from"], "%Y-%m-%d %H:%M:%S").replace(microsecond=1)
to_time = datetime.strptime(input_time["to"], "%Y-%m-%d %H:%M:%S").replace(microsecond=1)

prometheus = PrometheusSource(url="http://10.107.133.165:9090/")
extractor = MetricsExtractor()

save_metrics(selected_metrics, from_time, to_time, prometheus, extractor, experiment)

In [ ]:
def load_metrics(metric_name, experiment):
    df = pd.read_csv(f"metrics/{experiment}/{metric_name}.csv")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

players_data = load_metrics("mc_players_online_local", experiment)
tps_data = load_metrics("mc_tps", experiment)
cpu_data = load_metrics("container_cpu_usage_seconds_total", experiment)


#print dtypes
print(tps_data.dtypes)


In [ ]:

# primary_axis = AxisConfig(
#     labels=["Server 0 Players"],
#     ylabel="Players",
#     ylim=(0, 10),
#     plot_kwargs=[
#         {"color": "red", "linestyle": "-"},
#     ]
# )  

primary_axis = AxisConfig(
    labels=["Server 0 CPU"],
    ylabel="CPU Usage (\%)",
    ylim=(0, 4),
    plot_kwargs=[
        {"color": "red", "linestyle": "-"},
    ]
)

secondary_axis = AxisConfig(
    labels=["Server 0 TPS"],
    ylabel="TPS",
    ylim=(0, 20),
    plot_kwargs=[
        {"color": "red", "linestyle": "--"},
    ]
)

common_conf = CommonPlotConfig(
    title="CPU Usage and TPS",
    figsize=(15, 6),
    show_legend=True,
    legend_kwargs={"loc": "upper center"},
    tight_layout=True,
    grid=True,
    grid_minor=False,
    minor_ticks=False,
    # grid_kwargs={"linestyle": "-"},
    # minor_grid_kwargs={"linestyle": ":"},
    time_unit='s'
)

conf = PlotConfig(
    common=common_conf,
    primary_axis=primary_axis,
    secondary_axis=secondary_axis
)

#filter the data so that only timestamp and value are kept
cpu_data = cpu_data[["timestamp", "value"]]
players_data = players_data[["timestamp", "value"]]
tps_data = tps_data[["timestamp", "value"]]

plot_df([cpu_data], [tps_data], conf)

In [ ]:
def _is_function_query(query: str) -> bool:
    """
    Check if the query contains Prometheus functions.

    Args:
        query: The query string to check

    Returns:
        bool: True if the query contains functions, False otherwise
    """
    # Pattern to match functions like rate(), sum(), etc.
    function_pattern = r'[a-zA-Z_:][a-zA-Z0-9_:]*\('
    return bool(re.findall(function_pattern, query))

In [ ]:
_is_function_query("sum by(chunk_owner) (mc_player_location)")